In [2]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from keras.utils.vis_utils import plot_model
import itertools
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [2]:
%matplotlib inline
n_set = np.arange(len(y_train))
sample = np.random.choice(a=n_set, size=10000)
x_subset = x_train[sample]
y_subset = y_train[sample]

bounds = np.array([[0.001, 10]])

def f(lr):
    print("Learning rate:", lr)
    model = Sequential()

    model.add(Flatten())
    model.add(Dense(15, input_dim = 784, activation='sigmoid'))
    model.add(Dense(10, activation='sigmoid'))
    sgd=SGD(lr=lr)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

    history = model.fit(x_subset, y_subset, 
          batch_size=32, epochs=10, verbose=0, validation_data = (x_test, y_test))
    return -history.history['val_acc'][-1]

X_init = np.array([[0.01], [5]])
y1 = f(X_init[0])
y2 = f(X_init[1])
Y_init = np.array([[y1],[y2]])

W0724 16:04:22.833902 140240012580672 deprecation_wrapper.py:119] From /home/aa710/crowd_nets/jupyterenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 16:04:22.879026 140240012580672 deprecation_wrapper.py:119] From /home/aa710/crowd_nets/jupyterenv/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0724 16:04:22.882224 140240012580672 deprecation_wrapper.py:119] From /home/aa710/crowd_nets/jupyterenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 16:04:22.897036 140240012580672 deprecation_wrapper.py:119] From /home/aa710/crowd_nets/jupyterenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Ple

Learning rate: [0.01]


W0724 16:04:23.096740 140240012580672 deprecation.py:323] From /home/aa710/crowd_nets/jupyterenv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0724 16:04:23.162917 140240012580672 deprecation_wrapper.py:119] From /home/aa710/crowd_nets/jupyterenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Learning rate: [5.]


In [16]:
from scipy.stats import norm

def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.5):
    ''' Computes the EI at points X based on existing samples X_sample and Y_sample using a Gaussian process surrogate model. Args: X: Points at which EI shall be computed (m x d). X_sample: Sample locations (n x d). Y_sample: Sample values (n x 1). gpr: A GaussianProcessRegressor fitted to samples. xi: Exploitation-exploration trade-off parameter. Returns: Expected improvements at points X. '''
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, X_sample.shape[1])
    
    # Needed for noise-based model,
    # otherwise use np.max(Y_sample).
    # See also section 2.4 in [...]
    mu_sample_opt = np.max(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

def upper_confidence(X, X_sample, Y_sample, gpr, xi=0.1):
    mu, sigma = gpr.predict(X, return_std=True)
    up = mu -xi * sigma
    return up

In [4]:
from scipy.optimize import minimize

def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    ''' Proposes the next sampling point by optimizing the acquisition function. Args: acquisition: Acquisition function. X_sample: Sample locations (n x d). Y_sample: Sample values (n x 1). gpr: A GaussianProcessRegressor fitted to samples. Returns: Location of the acquisition function maximum. '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None
    
    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)
    
    # Find the best optimum by starting from n_restart different random points.
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')        
        if res.fun < min_val:
            min_val = res.fun[0]
            min_x = res.x           
            
    return min_x.reshape(-1, 1)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern
from bayesian_optimization_util import plot_approximation, plot_acquisition

noise=0.001
# Gaussian process with Mat??rn kernel as surrogate model
m52 = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5)
gpr = GaussianProcessRegressor(kernel=m52, alpha=noise**2)
X = np.arange(bounds[:, 0], bounds[:, 1], 0.01).reshape(-1, 1)
# Initialize samples
X_sample = X_init
Y_sample = Y_init

# Number of iterations
n_iter = 15

plt.figure(figsize=(12, n_iter * 3))
plt.subplots_adjust(hspace=0.4)

for i in range(n_iter):
    print("Training model", i+1)
    # Update Gaussian process with existing samples
    gpr.fit(X_sample, Y_sample)

    # Obtain next sampling point from the acquisition function (expected_improvement)
    X_next = propose_location(upper_confidence, X_sample, Y_sample, gpr, bounds)
    
    # Obtain next noisy sample from the objective function
    Y_next = f(X_next[0])
    
    # Plot samples, surrogate function, noise-free objective and next sampling location
    plt.subplot(n_iter, 2, 2 * i + 1)
    mu, std = gpr.predict(X, return_std=True)
    plt.plot(X, mu, 'b-', lw=1, label='Surrogate function')
    plt.fill_between(X.ravel(), 
                     mu.ravel() + 1.96 * std, 
                     mu.ravel() - 1.96 * std, 
                     alpha=0.1) 
    plt.plot(X_sample, Y_sample, 'kx', mew=3, label='Noisy samples')
    if X_next:
        plt.axvline(x=X_next, ls='--', c='k', lw=1)
    plt.title(f'Iteration {i+1}')
    
    plt.subplot(n_iter, 2, 2 * i + 2)
    plot_acquisition(X, upper_confidence(X, X_sample, Y_sample, gpr), X_next, show_legend=i==0)
    
    # Add sample to previous samples
    X_sample = np.vstack((X_sample, X_next))
    Y_sample = np.vstack((Y_sample, Y_next))

Training model 1
Learning rate: [0.001]
Training model 2
Learning rate: [3.88467354]
